In [1]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn
## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [2]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

## Генерация датасета

In [3]:
iris = datasets.load_iris()

In [4]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                    test_size=0.3, random_state=0)

## Задание модели

In [6]:
classifier = linear_model.SGDClassifier(random_state=0, tol=1e-3)

## Генерация сетки

In [9]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [10]:
parameters_grid = {
    'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty': ['l1', 'l2'],
    'max_iter': np.arange(5, 10),
    'alpha': np.linspace(0.0001, 0.001, num=5),
}

In [13]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

## Подбор параметров и оценка качества

### Grid search

In [15]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring='accuracy', cv=cv)

In [16]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 5.43 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2...
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge

In [17]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=9,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=0, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [42]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


In [21]:
grid_cv.cv_results_

{'mean_fit_time': array([0.03951709, 0.00179956, 0.00168602, 0.0016983 , 0.00438313,
        0.00199165, 0.00169547, 0.00159569, 0.00199449, 0.00189476,
        0.00189507, 0.00328686, 0.00219417, 0.00180531, 0.00298815,
        0.00198956, 0.00189774, 0.00179403, 0.00169337, 0.00159633,
        0.00179434, 0.0013958 , 0.00149486, 0.00260496, 0.00188954,
        0.00169156, 0.00199857, 0.00169485, 0.0017911 , 0.00169625,
        0.00199647, 0.00239346, 0.00199502, 0.00209248, 0.00189559,
        0.00190113, 0.00180879, 0.00279613, 0.00199513, 0.00180631,
        0.00169768, 0.0018779 , 0.0019913 , 0.00218279, 0.0022989 ,
        0.00188291, 0.00188956, 0.00189464, 0.00199807, 0.00169275,
        0.00170147, 0.0017977 , 0.00188868, 0.0015938 , 0.00208895,
        0.00175054, 0.00179234, 0.00149598, 0.00169697, 0.00169396,
        0.0018949 , 0.00149593, 0.00169537, 0.00169694, 0.00201705,
        0.00190217, 0.00209746, 0.00179687, 0.00189593, 0.00209441,
        0.00169454, 0.00188565,

## Randomized grid search

In [22]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring='accuracy', cv=cv, n_iter=20,
                                                       random_state=0)

In [23]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 485 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   error_score=nan,
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n_jobs=None,
                                           penal...
                                           verbose=0, warm_start=False),
                   iid='deprecated', n_iter=20, n_jobs=None,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.00

In [25]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}


In [31]:
#!/usr/bin/python

from datetime import datetime


def control_sum(str):
    return sum(map(ord, list(str)))


def check(reply):
    def _is_number(str):
        try:
            int(str)
            return True
        except ValueError:
            return False

    if "Control sum:" not in reply:
        return False
    parts = reply.split("Control sum:")
    received_current_time = parts[0].strip()
    received_control_sum = parts[1].strip()
    if not _is_number(received_control_sum):
        return False
    else:
        received_control_sum = int(received_control_sum)
    expected_control_sum = control_sum(received_current_time)
    return expected_control_sum == received_control_sum


current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)
print("Control sum: " + str(control_sum(current_time)))

print(check(current_time + '\n' + "Control sum: " + str(control_sum(current_time))))

2020-05-04 12:38:07
Control sum: 944
True


In [43]:
parameters_grid = {
    'penalty': ['l1', 'l2']
}

grid_search_cv = model_selection.GridSearchCV(classifier, parameters_grid, cv=cv, scoring='accuracy')

In [44]:
%%time
grid_search_cv.fit(train_data, train_labels)

Wall time: 66.9 ms


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2', power_t=0.5, random_state=0,
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'penalty': ['l1', 'l2']}, pre_dispatch='2*n_jobs',
             refit=True, retu

In [46]:
print(grid_search_cv.best_score_)
print(grid_search_cv.best_params_)

0.8619047619047618
{'penalty': 'l1'}


In [47]:
grid_search_cv.cv_results_

{'mean_fit_time': array([0.00228877, 0.00209432]),
 'std_fit_time': array([0.00045562, 0.00082966]),
 'mean_score_time': array([0.00019963, 0.00029902]),
 'std_score_time': array([0.00039926, 0.00045677]),
 'param_penalty': masked_array(data=['l1', 'l2'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'penalty': 'l1'}, {'penalty': 'l2'}],
 'split0_test_score': array([0.71428571, 0.71428571]),
 'split1_test_score': array([0.9047619 , 0.71428571]),
 'split2_test_score': array([0.95238095, 0.85714286]),
 'split3_test_score': array([1., 1.]),
 'split4_test_score': array([1.        , 0.85714286]),
 'split5_test_score': array([0.95238095, 0.57142857]),
 'split6_test_score': array([0.71428571, 0.9047619 ]),
 'split7_test_score': array([0.61904762, 0.85714286]),
 'split8_test_score': array([1.        , 0.76190476]),
 'split9_test_score': array([0.76190476, 0.9047619 ]),
 'mean_test_score': array([0.86190476, 0.81428571]),
 'std_test_score': 

***То есть GridSearchCV делает полный перебор по всей сетке параметров. Строится k-фолдов для каждого параметра в сетке. И для всех фолдов высчитывается accuracy(или другая метрика)***